In [1]:
import whisperx
import gc 

device = "cuda" 
audio_file = "audio1_5.mp3"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("whisper-ct2", device, compute_type=compute_type)

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

/opt/conda/envs/whisperx/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/whisperx/lib/python3.8/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures


ValueError: Invalid model size 'whisper-ct2', expected one of: tiny.en, tiny, base.en, base, small.en, small, medium.en, medium, large-v1, large-v2, large-v3, large, distil-large-v2, distil-medium.en, distil-small.en

In [6]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/4f/e9/4fe94fab3c6a8bf97c3289fc6feb023e1504259a53971018d4b19f2f7ed06185/69f74147e3334731bc3a76048724833325d2ec74642fb52620eda87352e3d4f1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.bin%3B+filename%3D%22model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1723886031&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzg4NjAzMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzRmL2U5LzRmZTk0ZmFiM2M2YThiZjk3YzMyODlmYzZmZWIwMjNlMTUwNDI1OWE1Mzk3MTAxOGQ0YjE5ZjJmN2VkMDYxODUvNjlmNzQxNDdlMzMzNDczMWJjM2E3NjA0ODcyNDgzMzMyNWQyZWM3NDY0MmZiNTI2MjBlZGE4NzM1MmUzZDRmMT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=cVkspt4h5v%7EIaF7gWHZuInClqkvrq-xm6eSFuESsdnthV7XcMOi2wXG2RULbG3R8e47QTaIelqfNakKblB%7El6BfKc5TE3j70KBAQnISBA4o1fv5Qm-LXiDbj3SQz3pUkvplD8WgvUVRWAwirOVq%7EQWMxSYyM%7Ej4vbgm06P

KeyboardInterrupt: 

In [2]:
import logging
import os

from typing import Optional

import huggingface_hub
import requests

from tqdm.auto import tqdm

In [3]:
_MODELS = (
    "tiny.en",
    "tiny",
    "base.en",
    "base",
    "small.en",
    "small",
    "medium.en",
    "medium",
    "large-v1",
    "large-v2",
)

In [4]:
def download_model(
    size: str,
    output_dir: Optional[str] = None,
    local_files_only: bool = False,
    cache_dir: Optional[str] = None,
):
    """Downloads a CTranslate2 Whisper model from the Hugging Face Hub.

    The model is downloaded from https://huggingface.co/guillaumekln.

    Args:
      size: Size of the model to download (tiny, tiny.en, base, base.en, small, small.en,
        medium, medium.en, large-v1, or large-v2).
      output_dir: Directory where the model should be saved. If not set, the model is saved in
        the cache directory.
      local_files_only:  If True, avoid downloading the file and return the path to the local
        cached file if it exists.
      cache_dir: Path to the folder where cached files are stored.

    Returns:
      The path to the downloaded model.

    Raises:
      ValueError: if the model size is invalid.
    """
    if size not in _MODELS:
        raise ValueError(
            "Invalid model size '%s', expected one of: %s" % (size, ", ".join(_MODELS))
        )

    repo_id = "guillaumekln/faster-whisper-%s" % size

    allow_patterns = [
        "config.json",
        "model.bin",
        "tokenizer.json",
        "vocabulary.txt",
    ]

    kwargs = {
        "local_files_only": local_files_only,
        "allow_patterns": allow_patterns,
        "tqdm_class": disabled_tqdm,
    }

    if output_dir is not None:
        kwargs["local_dir"] = output_dir
        kwargs["local_dir_use_symlinks"] = False

    if cache_dir is not None:
        kwargs["cache_dir"] = cache_dir


    try:
        return huggingface_hub.snapshot_download(repo_id, **kwargs)
    except (
        huggingface_hub.utils.HfHubHTTPError,
        requests.exceptions.ConnectionError,
    ) as exception:
        logger = get_logger()
        logger.warning(
            "An error occured while synchronizing the model %s from the Hugging Face Hub:\n%s",
            repo_id,
            exception,
        )
        logger.warning(
            "Trying to load the model directly from the local cache, if it exists."
        )

        kwargs["local_files_only"] = True
        return huggingface_hub.snapshot_download(repo_id, **kwargs)


In [5]:
model = WhisperModel.from_pretrained("whisper-ct2")

NameError: name 'WhisperModel' is not defined

In [ ]:
model.push_to_hub("nielsr/my-awesome-bert-model")

In [8]:
%pwd

'/workspaces/endtoend/research'

In [9]:
import os
os.chdir("/workspaces/endtoend")

In [11]:
import ctranslate2
import librosa
import transformers

# Load and resample the audio file.
audio, _ = librosa.load("audio.wav", sr=16000, mono=True)

# Compute the features of the first 30 seconds of audio.
processor = transformers.WhisperProcessor.from_pretrained("openai/whisper-tiny")
inputs = processor(audio, return_tensors="np", sampling_rate=16000)
features = ctranslate2.StorageView.from_array(inputs.input_features)

# Load the model on CPU.
model = ctranslate2.models.Whisper("whisper-ct2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
model.push_to_hub("vladimir7542/ctranslate")

AttributeError: 'ctranslate2._ext.Whisper' object has no attribute 'push_to_hub'

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path="whisper-ct2",
    repo_id="vladimir7542/faster-whisper",
    repo_type="space",
)